## R3 To R1 Point Attractor, S2 Damping, R1 Box Avoidance

In [1]:
using PBDS, StaticArrays, LinearAlgebra, GeometryBasics, Rotations, BenchmarkTools

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-12538585212149344916\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-4368257654915171814\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-12538585212149344916\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

## Setup

### Point Distance Attractor

In [2]:
PBDS.metric_chart(xn, task::Attractor{<:DistanceFromPoint{R3,R1,S}}, CN::Chart{1,R1}) where S = 
    default_metric(xn, task, CN)
PBDS.potential_chart(xn, task::Attractor{<:DistanceFromPoint{R3,R1,S}}, CN::Chart{1,R1}) where S = xn[1]^2
PBDS.dissipative_forces_chart(xn, vn, task::Attractor{<:DistanceFromPoint{R3,R1,S}}, CN::Chart{1,R1}) where S =  -5*vn
PBDS.weight_metric_chart(xn, vn, task::Attractor{<:DistanceFromPoint{R3,R1,S}}, CN::Chart{1,R1}) where S = 
    default_weight_metric(xn, vn, task, CN)

### Angular Damping Around Goal

In [3]:
PBDS.default_coord_rep(::Damping{<:AngularPositionAroundPoint{R3,S2,S}}) where S = EmbRep()
PBDS.codomain_coord_rep(::AngularPositionAroundPoint{R3,S2,S}) where S = EmbRep()
PBDS.metric_emb(xne, task::Damping{<:AngularPositionAroundPoint{R3,S2,S}}) where S =
    default_metric(xne, task)
PBDS.potential_emb(xne, task::Damping{<:AngularPositionAroundPoint{R3,S2,S}}) where S = 0.
PBDS.dissipative_forces_emb(xne, vne, task::Damping{<:AngularPositionAroundPoint{R3,S2,S}}) where S = -2*vne
PBDS.weight_metric_emb(xne, vne, task::Damping{<:AngularPositionAroundPoint{R3,S2,S}}) where S =
    default_weight_metric(xne, vne, task)

### Box Avoidance

In [4]:
function PBDS.metric_chart(xn, task::CollisionAvoidance{<:DistanceSphereToBox{R3,R1,S}}, CN::Chart{1,R1}) where S
    ψx = exp(1.e1 / xn[1]^2)
    G = SMatrix{1,1,eltype(xn)}([ψx])
end
PBDS.potential_chart(xn, task::CollisionAvoidance{<:DistanceSphereToBox{R3,R1,S}}, CN::Chart{1,R1}) where S = 0.
PBDS.dissipative_forces_chart(xn, vn, task::CollisionAvoidance{<:DistanceSphereToBox{R3,R1,S}}, CN::Chart{1,R1}) where S = 0*vn
function PBDS.weight_metric_chart(xn, vn, task::CollisionAvoidance{<:DistanceSphereToBox{R3,R1,S}}, CN::Chart{1,R1}) where S
    offset_distance = 5.
    λ = (xn[1] < offset_distance && vn[1] < 0.) ? 1. : 0.
    W = SMatrix{1,1,eltype(xn)}(I)*λ
end

In [5]:
xm_goal = SA[-6., 6., 0.]
M = R3
CM = Chart{1,M}()
tasks, CNs = TaskList(), ChartList()

N = R1
push!(tasks, Attractor(DistanceFromPoint{M,N}(xm_goal)))
push!(CNs, Chart{1,N}())

N = S2
push!(tasks, Damping(AngularPositionAroundPoint{M,N}(xm_goal)))
push!(CNs, Chart{SterProjSouth,N}())

N = R1
CN = Chart{1,N}()
radius = 0.5
center = SA[-3., 3., 0.]
widths = SA[3., 3., 2.]
corner = @. center - widths/2
rotation = RotXYZ(0.5, 0.5, 0.5)
push!(tasks, CollisionAvoidance(DistanceSphereToBox{M,N}(radius, Rect(Vec(corner), Vec(widths)), rotation)))
push!(CNs, CN)

3-element Array{Chart,1}:
 Chart{1,ℝ{1}}()
 Chart{SterProjSouth,𝕊{2}}()
 Chart{1,ℝ{1}}()

## Point Acceleration

In [6]:
# Initial state
xm = SA[1., 1., 1.]
vm = SA[-1., -1., -1.]
σxddot, = multiple_task_acceleration(xm, vm, tasks, CM, CNs)

LoadError: LAPACKException(2)

## Single Trajectory Test

In [7]:
Time = 10
dt = 0.01

traj = propagate_tasks(xm, vm, tasks, CM, CNs, Time, dt)
traj.xm[end]

LoadError: LAPACKException(2)

## Multiple Trajectories

In [8]:
using Makie, Observables, ProgressMeter, VectorizedRoutines

In [9]:
Time = 15
dt = 0.05
xlim, Δx = 1., 2.
vlim, Δv = 2., 4.

xm0 = Array{Any}(undef, 3)
vm0 = Array{Any}(undef, 3)

xm0[1], xm0[2], xm0[3] = Matlab.meshgrid(-xlim:Δx:xlim, -xlim:Δx:xlim, -xlim:Δx:xlim)
vm0[1], vm0[2], vm0[3] = Matlab.meshgrid(-vlim:Δv:vlim, -vlim:Δv:vlim, -vlim:Δv:vlim)

for a in (xm0, vm0), i in 1:3
    a[i] = reshape(a[i], length(a[i]))
end

nx = length(xm0[1])
nv = length(vm0[1])
ix, iv = Matlab.meshgrid(1:nx, 1:nv)
ix = reshape(ix, length(ix))
iv = reshape(iv, length(iv))

ntraj = nx*nv
trajs = Array{Any}(undef, ntraj)
xm0 = [xm0[1]'; xm0[2]'; xm0[3]']
vm0 = [vm0[1]'; vm0[2]'; vm0[3]']
p = Progress(ntraj)

Threads.@threads for i in 1:ntraj
    trajs[i] = propagate_tasks(SA[xm0[:,ix[i]]...], SA[vm0[:,iv[i]]...], tasks, CM, CNs, Time, dt)
    next!(p)
end

LoadError: TaskFailedException:
LAPACKException(2)
Stacktrace:
 [1] chklapackerror at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.5/LinearAlgebra/src/lapack.jl:38 [inlined]
 [2] gesdd!(::Char, ::Array{Float64,2}) at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.5/LinearAlgebra/src/lapack.jl:1650
 [3] _svd! at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.5/LinearAlgebra/src/svd.jl:105 [inlined]
 [4] svd!(::Array{Float64,2}; full::Bool, alg::LinearAlgebra.DivideAndConquer) at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.5/LinearAlgebra/src/svd.jl:98
 [5] #svd#85 at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.5/LinearAlgebra/src/svd.jl:158 [inlined]
 [6] pinv(::Array{Float64,2}; atol::Float64, rtol::Float64) at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.5/LinearAlgebra/src/dense.jl:1356
 [7] pinv(::Array{Float64,2}) at /buildworker/worker/package_lin

In [10]:
fxx(i) = [trajs[j].xm[i][1] for j = 1:ntraj]
fxy(i) = [trajs[j].xm[i][2] for j = 1:ntraj]
fxz(i) = [trajs[j].xm[i][3] for j = 1:ntraj]
iobs = Observable(1)

ax_size, plot_size = 5, 800
limits = FRect3D((-ax_size-3, -ax_size+3, -ax_size), (2*ax_size, 2*ax_size, 2*ax_size))
scene = Scene(resolution = (plot_size, plot_size))
Makie.scatter!(scene, lift(i -> fxx(i), iobs), lift(i -> fxy(i), iobs), lift(i -> fxz(i), iobs), markersize = ax_size/20, color = :blue, limits = limits)
Makie.scatter!(scene, xm_goal', markersize = ax_size/20, color = :darkgreen)
rect = Rect(Vec(-widths./2), Vec(widths))
rect_mesh = mesh!(rect, color = :orange)
q = UnitQuaternion(rotation)
qrot = Quaternion(q.w, q.x, q.y, q.z)
Makie.rotate!(rect_mesh[end], qrot)
Makie.translate!(rect_mesh[end], center)
for i = 1:ntraj
    Makie.lines!(scene, getindex.(trajs[i].xm,1), getindex.(trajs[i].xm,2), getindex.(trajs[i].xm,3), color = :purple)
end

Makie.xlabel!(scene, "x")
Makie.ylabel!(scene, "y")
axis = scene[Axis]
axis.showaxis = false
display(scene)

LoadError: UndefRefError: access to undefined reference

In [11]:
function record_scene(scene, filename, iobs, N, framerate=60)
    p = Progress(N)
    record(scene, filename, 1:N) do i
        iobs[] = i
        rotate_cam!(scene, 0.01, 0., 0.)
        next!(p)
    end
    display("text/html", html_video(filename))
end

filename = "R3ToR1xS2PointDistanceAttractorXDamping_R1BoxAvoidance.mp4"
record_scene(scene, filename, iobs, length(trajs[1].xm))

LoadError: UndefRefError: access to undefined reference